# Introduction

The idea behind nornir_tests is to allow for adding tests against the task run and its results.  Many times, a task can run and get some output but that doesn't mean it is successful.  One option is to add additional checks or some conditional logic to deal with this.  Another is to add tests against the data returned from a task.  That is what nornir_tests is for.

Typical nornir initialization is done first.

In [6]:
from nornir_utils.plugins.tasks.data import echo_data
from nornir_utils.plugins.functions import print_result
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)

Next the TestsProcessor is appended to the processors list and the inventory is shown.

In [7]:
from nornir_tests.plugins.processors import TestsProcessor

nr.processors.append(TestsProcessor())

print(nr.inventory.hosts)

{'test': Host: test}


Using more typical nornir methodology the process of running a task and verifying certain aspects about it can be accomplished a many different ways.  The example case is to simply run an echo_data task and validate that the result is changed and it completed within 10 seconds.

In [8]:
import time

def grouped_task(task, *args, **kwargs):
    t0 = time.time()
    result = task.run(task=echo_data, *args, **kwargs)
    t1 = time.time()

    # one way to validate, kinda clunky
    if t1 - t0 > 10:
        result.failed = True
    
    if result.changed != True:
        result.failed = True

    # and to get the above into results either return another Result or append tests
    result.tests = [
        f'changed: changed=True - {result.changed == True}',
        f'timing: 0 < {t1 - t0} < {sys.maxsize} - {t1 - t0 <= 10}'
    ]
    
result = nr.run(task=grouped_task, x=1, y=1)
print_result(result)

grouped_task********************************************************************
* test ** changed : False ******************************************************
vvvv grouped_task ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
Traceback (most recent call last):
  File "/home/patrick/dev/nornir_tests/.venv/lib/python3.8/site-packages/nornir/core/task.py", line 98, in start
    r = self.task(self, **self.params)
  File "<ipython-input-8-2b81df11fe9b>", line 13, in grouped_task
    result.failed = True
AttributeError: can't set attribute

---- echo_data ** changed : False ---------------------------------------------- INFO
{'x': 1, 'y': 1}
^^^^ END grouped_task ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Unfortunately we cannot change the failed state at this point.  So perhaps we just conditionally 

In [9]:
nr.data.reset_failed_hosts()

from nornir_tests.plugins.tests import test_regexp, test_until

result = nr.run(
    task=echo_data, 
    x=1, y=1,
    tests=[
        test_regexp(regexp='123'),
        test_regexp(regexp="\'x\': 1"),
        test_until(delay=1, retries=1)
    ]
)

print_result(result, vars=['result', 'tests'])

echo_data***********************************************************************
* test ** changed : False ******************************************************
vvvv echo_data ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{'x': 1, 'y': 1}
^^^^ END echo_data ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [11]:
for r in result.values():
    print(r[0].tests)

'regexp: 123 did not match result - FAILED'
"regexp: 'x': 1 matched 'x': 1 in result - PASSED"
'until: succeeded after 0.00032067298889160156 seconds - PASSED'

